# Load data

In [ ]:
from datetime import datetime
from collections import defaultdict

from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData
from vnpy.trader.utility import extract_vt_symbol

db = get_database()

In [ ]:
# Load data
vt_symbols = ["BTCUSDT_SWAP_BINANCE.GLOBAL", "ETHUSDT_SWAP_BINANCE.GLOBAL"]

history: dict[datetime, dict[str, BarData]] = defaultdict(dict)

for vt_symbol in vt_symbols:
    symbol, exchange = extract_vt_symbol(vt_symbol)

    bars: list[BarData] = db.load_bar_data(
        symbol=symbol,
        exchange=exchange,
        interval=Interval.MINUTE,
        start=datetime(2025, 8, 1, tzinfo=DB_TZ),
        end=datetime(2025, 9, 10, tzinfo=DB_TZ)
    )

    for bar in bars:
        history[bar.datetime][vt_symbol] = bar


# Initialize table

In [ ]:
# Choose which table to test
from vnpy_novastrategy import (
    LiveDataTable as DataTable,
    # BacktestingDataTable as DataTable
)

# Create table
table = DataTable(
    vt_symbols=vt_symbols,
    size=200,
    interval=Interval.MINUTE,
    extra_fields=["active_volume", "active_turnover", "trade_count"]
)


In [ ]:
# Add simple feature
table.add_feature("range", "high_price - low_price")

In [ ]:
# Add feature with parameters
for fast_window, slow_window in [
    (5, 25),
    (10, 50),
    (20, 100)
]:
    table.add_feature(
        name=f"ma_gap_{fast_window}_{slow_window}",
        expression=f"(ts_mean(close_price, {fast_window}) / ts_mean(close_price, {slow_window}) - 1) * 100"
    )

In [ ]:
# Add feature with complex expression
mfi_period = 15
neutral_period = 20

_mfi = f"ta_mfi(high_price, low_price, close_price, volume, {mfi_period})"
_mfi_mean = f"ts_mean({_mfi}, {neutral_period})"
_mfi_std = f"ts_std({_mfi}, {neutral_period})"
_mfi_zscore = f"({_mfi} - {_mfi_mean}) / {_mfi_std}"
expression = f"rank({_mfi_zscore})"
print(expression)

table.add_feature("ranked_mfi", expression)

In [ ]:
# Add feature with complex expressions
mfi_period = 15
neutral_period = 20

_mfi = f"ta_mfi(high_price, low_price, close_price, volume, {mfi_period})"
_mfi_mean = f"ts_mean({_mfi}, {neutral_period})"
_mfi_std = f"ts_std({_mfi}, {neutral_period})"
_mfi_zscore = f"({_mfi} - {_mfi_mean}) / {_mfi_std}"
expression = f"rank({_mfi_zscore})"
print(expression)

table.add_feature("ranked_mfi", expression)

In [ ]:
# Add feature with intermediate variables
short_window = 5
long_window = 20
signal_window = 50

table.add_feature("short_ma", f"ts_mean(close_price, {short_window})")
table.add_feature("long_ma", f"ts_mean(close_price, {long_window})")
table.add_feature("buy_signal_mask", "ts_greater_than(short_ma, long_ma) * ts_greater_than(close_price, short_ma)")
table.add_feature("buy_proportion", f"ts_sum(buy_signal_mask, {signal_window}) / {signal_window}")
table.add_feature("buy_signal_proportion_rank", "rank(buy_proportion)")

In [ ]:
# Add feature with parameters
for rsi_window in [15, 20, 25]:
    for rsi_threshold in range(10, 80, 10):
        name = f"rsi_above_threshold_{rsi_window}_{rsi_threshold}"

        _rsi = f"ta_rsi(close_price, {rsi_window})"
        _rsi_above_threshold = f"ts_greater_than({_rsi}, {rsi_threshold})"
        expression = f"cs_sum({_rsi_above_threshold}) / cs_count(close_price)"

        table.add_feature(name, expression)


In [ ]:
# Add feature with cross-sectional zscore
table.add_feature("ma20_ma60", "cs_zscore(ts_mean(close_price, 20) / ts_mean(close_price, 60))")
table.add_feature("ma20_ma120", "cs_zscore(ts_mean(close_price, 20) / ts_mean(close_price, 120))")
table.add_feature("rsi_14", "cs_zscore(ta_rsi(close_price, 14))")

table.add_feature("label", "cs_zscore(ts_delay(close_price, -20) / close_price - 1)")

# For live trading usage

In [ ]:
bars

In [ ]:
# Update bars into table
for bars in history.values():
    table.update_bars(bars)

In [ ]:
# Get latest dataframe
df = table.get_df()
df.tail(20)

# For backtesting usage

In [ ]:
# Update entire history into table
table.update_history(list(history.values()))

In [ ]:
# Update bars into table
for bars in history.values():
    table.update_bars(bars)

In [ ]:
# Get latest dataframe
df = table.get_df()
df.tail(100)